#ONE CLICK SETup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U ultralytics

In [3]:
from ultralytics import YOLO

In [ ]:
!pip install ultralytics opencv-python


#Code to load Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Code to extract data

In [ ]:
ls drive/MyDrive/AI_IMAGE_PROJECT/

Neural_Ocean_extracted/  Neural_Ocean.v3i.yolov11.zip


In [ ]:
import zipfile
import os

zip_file_path = 'drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean.v3i.yolov11.zip'
extraction_path = 'drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f'Successfully extracted {zip_file_path} to {extraction_path}')

Successfully extracted drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean.v3i.yolov11.zip to drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted


In [ ]:
ls drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted/

data.yaml  README.dataset.txt  README.roboflow.txt  test/  train/  valid/


#Dataset Drive: https://universe.roboflow.com/neural-ocean/neural_ocean


In [ ]:
ls drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted/

data.yaml  README.dataset.txt  README.roboflow.txt  test/  train/  valid/


#library's required: ultralytics


In [ ]:
!pip install -U ultralytics


#Analysis of Dataset

In [ ]:
# @title
import os

BASE_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted"

splits = ["train", "valid", "test"]

for split in splits:
    img_dir = f"{BASE_PATH}/{split}/images"
    lbl_dir = f"{BASE_PATH}/{split}/labels"

    num_images = len(os.listdir(img_dir))
    num_labels = len(os.listdir(lbl_dir))

    print(f"{split.upper()}: Images = {num_images}, Labels = {num_labels}")


TRAIN: Images = 3626, Labels = 3626
VALID: Images = 1000, Labels = 1000
TEST: Images = 501, Labels = 501


In [ ]:
# @title
import yaml

with open(f"{BASE_PATH}/data.yaml", "r") as f:
    data = yaml.safe_load(f)

class_names = data["names"]
num_classes = data["nc"]

print("Number of classes:", num_classes)
print("Classes:", class_names)


Number of classes: 15
Classes: ['Mask', 'can', 'cellphone', 'electronics', 'gbottle', 'glove', 'metal', 'misc', 'net', 'pbag', 'pbottle', 'plastic', 'rod', 'sunglasses', 'tire']


In [ ]:
# @title
from collections import defaultdict

class_count = defaultdict(int)

for split in ["train", "valid", "test"]:
    label_dir = f"{BASE_PATH}/{split}/labels"

    for file in os.listdir(label_dir):
        with open(os.path.join(label_dir, file), "r") as f:
            for line in f:
                cls_id = int(line.split()[0])
                class_count[cls_id] += 1

print("Class Distribution:\n")
for cls_id, count in sorted(class_count.items()):
    print(f"{class_names[cls_id]:15s}: {count}")


Class Distribution:

Mask           : 1572
can            : 163
cellphone      : 385
electronics    : 196
gbottle        : 484
glove          : 1261
metal          : 83
misc           : 257
net            : 744
pbag           : 1631
pbottle        : 1342
plastic        : 275
rod            : 37
sunglasses     : 18
tire           : 3211


In [ ]:
# @title
import cv2

image_sizes = []

for split in ["train", "valid", "test"]:
    img_dir = f"{BASE_PATH}/{split}/images"

    for img_name in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            h, w, _ = img.shape
            image_sizes.append((w, h))

widths = [w for w, h in image_sizes]
heights = [h for w, h in image_sizes]

print("Avg width:", sum(widths)//len(widths))
print("Avg height:", sum(heights)//len(heights))
print("Min resolution:", min(widths), "x", min(heights))
print("Max resolution:", max(widths), "x", max(heights))


Avg width: 416
Avg height: 416
Min resolution: 416 x 416
Max resolution: 416 x 416


In [ ]:
# @title
empty_labels = []

for split in ["train", "valid", "test"]:
    label_dir = f"{BASE_PATH}/{split}/labels"

    for file in os.listdir(label_dir):
        file_path = os.path.join(label_dir, file)
        if os.path.getsize(file_path) == 0:
            empty_labels.append(file_path)

print("Empty label files:", len(empty_labels))


Empty label files: 0


#YOLOv11 description: https://docs.ultralytics.com/models/yolo11/#key-features


| Model          | Size    | Speed       | Accuracy | GPU/CPU Use | Best For                    |
| -------------- | ------- | ----------- | -------- | ----------- | --------------------------- |
| **yolo11n.pt** | Nano    | 🚀 Fastest  | ⭐ Lowest | Very low    | Edge devices, quick tests   |
| **yolo11s.pt** | Small   | ⚡ Very fast | ⭐⭐       | Low         | Real-time apps              |
| **yolo11m.pt** | Medium  | ⚖ Balanced  | ⭐⭐⭐⭐     | Medium      | Best overall choice         |
| **yolo11l.pt** | Large   | 🐢 Slower   | ⭐⭐⭐⭐⭐    | High        | High accuracy tasks         |
| **yolo11x.pt** | X-Large | 🐌 Slowest  | ⭐⭐⭐⭐⭐⭐   | Very high   | Research / maximum accuracy |

#Training with pretrained


In [ ]:
from ultralytics import YOLO

DATA_YAML = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted/data.yaml"

#TODO: dont use pre-trained model, make it from scratch
model = YOLO("yolo11n.pt")

model.train(
    data=DATA_YAML,
    epochs=80,
    imgsz=416,
    batch=16,            # ⬅️ safer for CPU
    device="cpu",        # ✅ FIX
    workers=4,           # ⬅️ CPU-friendly
    optimizer="AdamW",
    lr0=0.002,
    cos_lr=True,
    patience=15,
    augment=True,

    project="/content/drive/MyDrive/AI_IMAGE_PROJECT/results",
    name="yolov11n_416_baseline"
)


Validation Report


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_baseline/weights/best.pt")
model.val()


code to Resume Training

In [ ]:
from ultralytics import YOLO

# Load last checkpoint
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_baseline/weights/last.pt")

# Resume training
model.train(
    resume=True
)


#Training without pretrained


In [ ]:
from ultralytics import YOLO

In [ ]:


DATA_YAML = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Neural_Ocean_extracted/data.yaml"

# Load YOLOv11 Nano ARCHITECTURE ONLY (no pretrained weights)
model = YOLO("yolo11n.yaml")

In [ ]:
model.info()

YOLO11n summary: 182 layers, 2,624,080 parameters, 2,624,064 gradients, 6.6 GFLOPs


(182, 2624080, 2624064, 6.614336)

In [ ]:
print(model.model)

1 to 27 epoches

In [ ]:
# @title
# Train the model FROM SCRATCH
model.train(
    data=DATA_YAML,

    # Training duration
    epochs=120,          # more epochs needed without pretraining

    # Image & batch settings
    imgsz=416,
    batch=16,
    device="cpu",
    workers=4,

    # Optimisation
    optimizer="AdamW",
    lr0=0.003,           # higher LR for training from scratch
    cos_lr=True,
    momentum=0.937,
    weight_decay=0.0005,

    # Regularisation & robustness
    augment=True,
    patience=25,

    # Saving results
    project="/content/drive/MyDrive/AI_IMAGE_PROJECT/results",
    name="yolov11n_416_from_scratch"
)

#if (Resume Training)

28 to 55

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

56 to 83

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

84 to 86

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

87 to 97


In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

98 to 106

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

107 to 120

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/last.pt")
model.train(resume=True)

#After Training (MANDATORY FOR REPORT)

In [ ]:
model = YOLO("/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/best.pt")
model.val()

#RUN

In [ ]:
# @title
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np

def take_photo():
    js_code = '''
    async function takePhoto() {
        const video = document.createElement('video');
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        document.body.appendChild(video);
        video.srcObject = stream;
        await video.play();

        await new Promise(resolve => setTimeout(resolve, 1000));

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        video.remove();

        return canvas.toDataURL('image/jpeg', 0.8);
    }
    takePhoto();
    '''

    data = eval_js(js_code)
    img = b64decode(data.split(',')[1])
    frame = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
    return frame


In [ ]:
# @title
from ultralytics import YOLO
from matplotlib import pyplot as plt

MODEL_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/best.pt"
model = YOLO(MODEL_PATH)

while True:
    print("📸 Capturing frame...")
    frame = take_photo()

    results = model.predict(
        source=frame,
        imgsz=416,
        conf=0.25,
        device="cpu",
        verbose=False
    )

    annotated = results[0].plot()

    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    if input("Press Enter for next frame or type 'q' to quit: ").lower() == 'q':
        break


#RUN 2

In [ ]:
ls drive/MyDrive/AI_IMAGE_PROJECT/

In [ ]:
VIDEO_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Plastic Waste In The Sea Stock Video.mp4"
MODEL_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/best.pt"
OUTPUT_DIR = "/content/drive/MyDrive/AI_IMAGE_PROJECT/video_results"

In [ ]:
from ultralytics import YOLO
import os

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load model
model = YOLO(MODEL_PATH)

# Run inference on video
results = model.predict(
    source=VIDEO_PATH,
    imgsz=416,
    conf=0.25,
    device="cpu",      # Colab CPU safe
    save=True,         # Save output video
    project=OUTPUT_DIR,
    name="yolov11_video_detection2",
    verbose=True
)


#RUN 3

In [5]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import os


In [6]:
ls drive/MyDrive/AI_IMAGE_PROJECT/

'Garbage underwater - zero wildlife.mp4'
 Neural_Ocean_extracted/
 Neural_Ocean.v3i.yolov11.zip
'Plastic Waste In The Sea Stock Video.mp4'
 results/
 video_results/


In [7]:
# VIDEO_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Plastic Waste In The Sea Stock Video.mp4"
VIDEO_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/Garbage underwater - zero wildlife.mp4"
MODEL_PATH = "/content/drive/MyDrive/AI_IMAGE_PROJECT/results/yolov11n_416_from_scratch/weights/best.pt"


In [8]:
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)

frame_count = 0


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # YOLO inference on single frame
    results = model(
        frame,
        imgsz=416,
        conf=0.25,
        device="cpu"
    )

    # Draw predictions
    annotated_frame = results[0].plot()

    # Show every Nth frame to avoid overload
    if frame_count % 1 == 0:   # change to 1 for every frame
        print(f"Frame: {frame_count}")
        cv2_imshow(annotated_frame)

cap.release()
